In [1]:
# ==============================
# 🔥 Simplified Binance Futures Testnet Trading Bot
# Works directly in Google Colab
# ==============================

# Install required library
!pip install requests

import requests
import time
import hmac
import hashlib
from urllib.parse import urlencode
import getpass
import logging

# -----------------------------
# CONFIGURATION
# -----------------------------
BASE_URL = "https://testnet.binancefuture.com"  # Futures Testnet
ORDER_ENDPOINT = "/fapi/v1/order"
BALANCE_ENDPOINT = "/fapi/v2/balance"

# Setup logging
logging.basicConfig(
    filename='trading_bot.log',
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s'
)

# -----------------------------
# AUTHENTICATION
# -----------------------------
print("Enter your Binance Testnet API credentials:")
API_KEY = getpass.getpass("API Key: ")
API_SECRET = getpass.getpass("API Secret: ")

headers = {"X-MBX-APIKEY": API_KEY}

def _timestamp():
    return int(time.time() * 1000)

def _sign(query_string):
    """Generate HMAC SHA256 signature."""
    return hmac.new(API_SECRET.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

# -----------------------------
# CORE API FUNCTIONS
# -----------------------------
def send_signed_request(http_method, url_path, payload=None):
    payload = payload or {}
    payload['timestamp'] = _timestamp()
    query = urlencode(payload)
    signature = _sign(query)
    query += "&signature=" + signature
    url = BASE_URL + url_path + "?" + query
    logging.info(f"REQUEST: {http_method} {url}")

    try:
        if http_method == "GET":
            response = requests.get(url, headers=headers)
        elif http_method == "POST":
            response = requests.post(url, headers=headers)
        elif http_method == "DELETE":
            response = requests.delete(url, headers=headers)
        else:
            raise ValueError("Unsupported HTTP method")
    except Exception as e:
        logging.error(f"Network error: {e}")
        print("❌ Network error:", e)
        return None

    if response.status_code != 200:
        logging.error(f"API Error: {response.text}")
        print("❌ API Error:", response.text)
        return None

    logging.info(f"Response: {response.text}")
    return response.json()

# -----------------------------
# ORDER FUNCTIONS
# -----------------------------
def place_market_order(symbol, side, quantity):
    params = {
        "symbol": symbol.upper(),
        "side": side.upper(),
        "type": "MARKET",
        "quantity": quantity
    }
    print(f"🚀 Placing MARKET {side.upper()} order for {quantity} {symbol} ...")
    return send_signed_request("POST", ORDER_ENDPOINT, params)

def place_limit_order(symbol, side, quantity, price):
    params = {
        "symbol": symbol.upper(),
        "side": side.upper(),
        "type": "LIMIT",
        "quantity": quantity,
        "price": price,
        "timeInForce": "GTC"
    }
    print(f"🚀 Placing LIMIT {side.upper()} order for {quantity} {symbol} at {price} USDT ...")
    return send_signed_request("POST", ORDER_ENDPOINT, params)

def get_balance():
    print("💰 Fetching account balance ...")
    return send_signed_request("GET", BALANCE_ENDPOINT)

# -----------------------------
# INTERACTIVE COMMANDS
# -----------------------------
while True:
    print("\n--- Binance Futures Testnet Bot ---")
    print("1️⃣ Market Order")
    print("2️⃣ Limit Order")
    print("3️⃣ Show Balance")
    print("4️⃣ Exit")

    choice = input("Enter choice (1-4): ").strip()

    if choice == "1":
        sym = input("Symbol (e.g., BTCUSDT): ").upper()
        side = input("Side (BUY/SELL): ").upper()
        qty = float(input("Quantity: "))
        resp = place_market_order(sym, side, qty)
        print(resp)

    elif choice == "2":
        sym = input("Symbol (e.g., BTCUSDT): ").upper()
        side = input("Side (BUY/SELL): ").upper()
        qty = float(input("Quantity: "))
        price = float(input("Limit price (USDT): "))
        resp = place_limit_order(sym, side, qty, price)
        print(resp)

    elif choice == "3":
        resp = get_balance()
        print(resp)

    elif choice == "4":
        print("👋 Exiting bot.")
        break

    else:
        print("❌ Invalid choice, please try again.")


Enter your Binance Testnet API credentials:
API Key: ··········
API Secret: ··········

--- Binance Futures Testnet Bot ---
1️⃣ Market Order
2️⃣ Limit Order
3️⃣ Show Balance
4️⃣ Exit
Enter choice (1-4): 1
Symbol (e.g., BTCUSDT): B
Side (BUY/SELL): SELL
Quantity: 100
🚀 Placing MARKET SELL order for 100.0 B ...


ERROR:root:API Error: {"code":-2014,"msg":"API-key format invalid."}


❌ API Error: {"code":-2014,"msg":"API-key format invalid."}
None

--- Binance Futures Testnet Bot ---
1️⃣ Market Order
2️⃣ Limit Order
3️⃣ Show Balance
4️⃣ Exit
Enter choice (1-4): 3


ERROR:root:API Error: {"code":-2014,"msg":"API-key format invalid."}


💰 Fetching account balance ...
❌ API Error: {"code":-2014,"msg":"API-key format invalid."}
None

--- Binance Futures Testnet Bot ---
1️⃣ Market Order
2️⃣ Limit Order
3️⃣ Show Balance
4️⃣ Exit
Enter choice (1-4): 4
👋 Exiting bot.
